In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import string

In [4]:
alphabetical_critics_URL = ['https://www.rottentomatoes.com/critics/authors?letter=' + i for i in string.ascii_lowercase]

In [ ]:
# First, we scrap all reviewers name and URL 

all_critics = {'critic_name':[], 'critic_URL':[]}
headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
for query in alphabetical_critics_URL:
    r = requests.get(query, headers=headers)
    if r.status_code != 200:
        print('problem while attempting to scrap :', query)
    else:
        soup = BeautifulSoup(r.content, 'html.parser')
        for critic in soup.select('.critic-authors__name'):
            all_critics['critic_name'].append(critic.get_text())
            all_critics['critic_URL'].append('https://www.rottentomatoes.com' + critic.get('href'))

In [ ]:
df_critics = pd.DataFrame(all_critics)
df_critics['critic_id'] = df_critics.index + 1

In [5]:
df_critics.head()

,critic_name,critic_URL,critic_id
0,Josefine A.,https://www.rottentomatoes.com/critic/josefine-a,1
1,Lysalex Hernández A.,https://www.rottentomatoes.com/critic/lysalex-...,2
2,Arun A.K,https://www.rottentomatoes.com/critic/arun-ak,3
3,Alex Abad-Santos,https://www.rottentomatoes.com/critic/alex-aba...,4
4,Davide Abbatescianni,https://www.rottentomatoes.com/critic/davide-a...,5


In [7]:
df_critics.shape

(3576, 3)

In [ ]:
# Export of all the critics name, URL and id.
df_critics.to_csv('data/scrapped_data/all_critics.csv', index=False)

In [3]:
# It is possible to start from here with the csv 'all_critics'
df_critics = pd.read_csv('data/scrapped_data/all_critics.csv')

In [5]:
# Second, we scrap the all the rates given by each reviewer

all_scores= {
            'critic_id' : [],
            'scrapping_status' : [],
            'critic_score': [],
            'critic_bool' : [],
            'RT_score': [],
            'movie_URL': [],
            'movie_title': [],
            'movie_year': []
            }

all_scores_pd = pd.DataFrame(all_scores)
all_scores_pd.to_csv('data_collected/all_scores.csv', index=False)

In [ ]:
headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
critic_processed = 0

for j in range(df_critics['critic_id'].max()):
    i = 1
    critic_id = j + 1
    critic_URL = df_critics.iloc[j,1]
    
    # we try that because not all critics have a 'movie' URL (only 'TV' for example) and because we don't know how many critics
    # have been written by this individual.
    movie_score_URL = critic_URL + '/movies?page=' + str(i)
    print(movie_score_URL)
    r = requests.get(movie_score_URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    if r.url[-2:] == 'tv' or soup.select("#review-table-body tr") == []:
        print('problem while attempting to scrap :', movie_score_URL)
        all_scores['critic_id'].append(critic_id)
        all_scores['scrapping_status'].append('nothing_found')
        all_scores['critic_score'].append(None)
        all_scores['RT_score'].append(None)
        all_scores['movie_URL'].append(None)
        all_scores['movie_title'].append(None)
        all_scores['movie_year'].append(None)
    else:
        scrap_status = True
        while scrap_status == True:
            soup = BeautifulSoup(r.content, 'html.parser')
            for movie in soup.select("#review-table-body tr"):
                try:
                    movie_data = movie.find_all('td')
                    all_scores['critic_id'].append(critic_id)
                    all_scores['scrapping_status'].append('ok') # means at_least_one_review_found
                    all_scores['critic_score'].append(movie_data[0].get_text().strip())

                    critic_logo = movie_data[0].find('span').get_attribute_list('class')[2]
                    if critic_logo == 'fresh':
                        all_scores['critic_bool'].append(1)
                    elif critic_logo == 'rotten':
                        all_scores['critic_bool'].append(0)
                    else:
                        all_scores['critic_bool'].append(None)

                    all_scores['RT_score'].append(movie_data[1].get_text().strip())
                    all_scores['movie_URL'].append(movie_data[2].find('a').get('href'))
                    movie_desc = movie_data[2].get_text().strip().replace('\n', '')
                    year = movie_desc[-5:-1]
                    title = movie_desc.replace('(' + year + ')', '')
                    all_scores['movie_year'].append(year)
                    all_scores['movie_title'].append(' '.join(title.split()))
                except:
                    pass
            # we move to the next page
            i +=1
            print(i)
            movie_score_URL = critic_URL + '/movies?page=' + str(i)
            print(movie_score_URL)
            r = requests.get(movie_score_URL, headers=headers)
            soup = BeautifulSoup(r.content, 'html.parser')
            if soup.select("#review-table-body tr")==[]:
                scrap_status = False
                critic_processed += 1
                if critic_processed % 10 == 1:
                    print(all_scores)
                    all_scores_pd = pd.DataFrame(all_scores)
                    all_scores_pd.to_csv('data_collected/all_scores.csv', index=False, mode='a', header=False)                 
                    all_scores= {'critic_id' : [],'scrapping_status' : [],'critic_score': [],'critic_bool' : [],
                                 'RT_score': [],#'RT_bool' : [],
                                 'movie_URL': [], 'movie_title': [], 'movie_year': []
                                }
                print('end of this critic')

https://www.rottentomatoes.com/critic/josefine-a/movies?page=1


In [3]:
df_all_scores = pd.read_csv('data/scrapped_data/all_scores.csv')

/Users/maurerhugo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_all_scores.shape

(871304, 8)

### Finally, we collected 871,304 reviews that can be used to build a collaborative filtering model (user/item rates matrix).